In [100]:
import os
from sqlalchemy import create_engine
from sqlalchemy import text

In [101]:
# Create fake database.

if os.path.exists("some.db"):
    os.remove("some.db")
e = create_engine("sqlite:///some.db")
with e.begin() as conn:
    conn.execute(
        text(
            """
        create table employee (
            emp_id integer primary key,
            emp_name varchar
        )
    """
        )
    )

    conn.execute(
        text(
            """
        create table employee_of_month (
            emp_id integer primary key,
            emp_name varchar
        )
    """
        )
    )

    conn.execute(
        text("insert into employee(emp_name) values (:name)"),
        [{"name": "spongebob"}, {"name": "sandy"}, {"name": "squidward"}],
    )


Normal DBAPI will be using following codes.

``` python
connection = ...

cursor = connection.cursor()
cursor.execute(...)
result = cursor.fetchall()

cursor.close()
connection.commit()
``` 

Setting future to be `True` will preventing SQLAlchemy v2.0 from using python2 and other deprecated feateures.

In [102]:
engine = create_engine("sqlite:///some.db", future=True, echo=True)
type(engine)

sqlalchemy.future.engine.Engine

In [103]:
connection = engine.connect()
connection

In [104]:
connection.connection.connection

In [105]:
stmt = text("select emp_id, emp_name from employee where emp_id=:emp_id")
result = connection.execute(stmt, {"emp_id": 2})

2023-06-23 20:22:03,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:22:03,723 INFO sqlalchemy.engine.Engine select emp_id, emp_name from employee where emp_id=?
2023-06-23 20:22:03,724 INFO sqlalchemy.engine.Engine [generated in 0.00128s] (2,)


In [106]:
# Result is like a cursor. Using first() will return the content and close the result.
row = result.first()

In [107]:
print(row)
print(row[1])
print(row.emp_name)
print(row['emp_name'])

(2, 'sandy')
sandy
sandy
sandy


In [108]:
print(row._mapping["emp_name"])
print(row._mapping.keys())
print(row._mapping.values())

sandy
RMKeyView(['emp_id', 'emp_name'])
ROMappingView({'emp_id': 2, 'emp_name': 'sandy'})


In [110]:
result = connection.execute(text("select * from employee"))
for emp_name, emp_id in result:
    print(f"employee id: {emp_id}  employee name: {emp_name}")
result.all()

2023-06-23 20:22:32,105 INFO sqlalchemy.engine.Engine select * from employee
2023-06-23 20:22:32,106 INFO sqlalchemy.engine.Engine [cached since 25.69s ago] ()
employee id: spongebob  employee name: 1
employee id: sandy  employee name: 2
employee id: squidward  employee name: 3


[]

Normally closing a connection will not actually close the connection. Instead it will put the connection back to the connection pool.

In [95]:
connection.close()

2023-06-23 20:20:00,934 INFO sqlalchemy.engine.Engine ROLLBACK


In [98]:
# Use context managers to make connections.
# Results will be gone when connection is closed.

with engine.connect() as connection:
    rows = connection.execute(text("select * from employee"))
    print(rows.all())

2023-06-23 20:20:33,610 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:20:33,611 INFO sqlalchemy.engine.Engine select * from employee
2023-06-23 20:20:33,611 INFO sqlalchemy.engine.Engine [cached since 37.32s ago] ()
[(1, 'spongebob'), (2, 'sandy'), (3, 'squidward')]
2023-06-23 20:20:33,612 INFO sqlalchemy.engine.Engine ROLLBACK


SQLAlchemy will not commit automatically. So `.commit()` will be needed.

In [74]:
with engine.connect() as connection:
    connection.execute(  
        text("insert into employee_of_month (emp_name) values (:emp_name)"),
        {"emp_name": "sandy"}
    )
    connection.commit()

2023-06-23 20:16:29,925 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:16:29,925 INFO sqlalchemy.engine.Engine insert into employee_of_month (emp_name) values (?)
2023-06-23 20:16:29,926 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ('sandy',)
2023-06-23 20:16:29,928 INFO sqlalchemy.engine.Engine COMMIT


`begin` will begin a transaction that will automatically commit or rollback at the end of the transaction. Normal connection will only automatically roll back.

In [75]:
with engine.begin() as connection:
    connection.execute(
        text("insert into employee_of_month (emp_name) values (:emp_name)"),
        {"emp_name": "squidward"},
    )

2023-06-23 20:16:30,988 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:16:30,989 INFO sqlalchemy.engine.Engine insert into employee_of_month (emp_name) values (?)
2023-06-23 20:16:30,989 INFO sqlalchemy.engine.Engine [cached since 1.064s ago] ('squidward',)
2023-06-23 20:16:30,990 INFO sqlalchemy.engine.Engine COMMIT


In [76]:
with engine.connect() as connection:
    with connection.begin():
        connection.execute(
            text("update employee_of_month set emp_name = :emp_name"),
            {"emp_name": "squidward"},
        )

2023-06-23 20:16:31,549 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:16:31,549 INFO sqlalchemy.engine.Engine update employee_of_month set emp_name = ?
2023-06-23 20:16:31,550 INFO sqlalchemy.engine.Engine [generated in 0.00039s] ('squidward',)
2023-06-23 20:16:31,551 INFO sqlalchemy.engine.Engine COMMIT


In [77]:
with engine.connect() as connection:
    with connection.begin():
        savepoint = connection.begin_nested()
        connection.execute(
            text("update employee_of_month set emp_name = :emp_name"),
            {"emp_name": "patrick"},
        )
        savepoint.rollback()

        with connection.begin_nested() as savepoint:
            connection.execute(
                text("update employee_of_month set emp_name = :emp_name"),
                {"emp_name": "spongebob"},
            )

2023-06-23 20:16:31,995 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:16:31,996 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_1
2023-06-23 20:16:31,996 INFO sqlalchemy.engine.Engine [no key 0.00045s] ()
2023-06-23 20:16:31,997 INFO sqlalchemy.engine.Engine update employee_of_month set emp_name = ?
2023-06-23 20:16:31,998 INFO sqlalchemy.engine.Engine [cached since 0.4483s ago] ('patrick',)
2023-06-23 20:16:31,999 INFO sqlalchemy.engine.Engine ROLLBACK TO SAVEPOINT sa_savepoint_1
2023-06-23 20:16:31,999 INFO sqlalchemy.engine.Engine [no key 0.00039s] ()
2023-06-23 20:16:32,000 INFO sqlalchemy.engine.Engine SAVEPOINT sa_savepoint_2
2023-06-23 20:16:32,000 INFO sqlalchemy.engine.Engine [no key 0.00049s] ()
2023-06-23 20:16:32,001 INFO sqlalchemy.engine.Engine update employee_of_month set emp_name = ?
2023-06-23 20:16:32,002 INFO sqlalchemy.engine.Engine [cached since 0.4522s ago] ('spongebob',)
2023-06-23 20:16:32,002 INFO sqlalchemy.engine.Engine RELEASE SAVEPOINT s

In [78]:
with engine.connect().execution_options(isolation_level="AUTOCOMMIT") as connection:
    connection.execute(
        text("insert into employee(emp_name) values (:name)"),
        {"name": "plankton"},
    )

2023-06-23 20:16:32,517 INFO sqlalchemy.engine.Engine BEGIN (implicit; DBAPI should not BEGIN due to autocommit mode)
2023-06-23 20:16:32,517 INFO sqlalchemy.engine.Engine insert into employee(emp_name) values (?)
2023-06-23 20:16:32,518 INFO sqlalchemy.engine.Engine [generated in 0.00090s] ('plankton',)
2023-06-23 20:16:32,520 INFO sqlalchemy.engine.Engine ROLLBACK using DBAPI connection.rollback(), DBAPI should ignore due to autocommit mode


In [79]:
with engine.connect() as connection:
    planktons_id = connection.execute(
        text("select emp_id from employee where emp_name=:name"),
        {"name": "plankton"}
    ).scalar()
    print(planktons_id)

2023-06-23 20:16:33,110 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-06-23 20:16:33,110 INFO sqlalchemy.engine.Engine select emp_id from employee where emp_name=?
2023-06-23 20:16:33,111 INFO sqlalchemy.engine.Engine [generated in 0.00077s] ('plankton',)
4
2023-06-23 20:16:33,111 INFO sqlalchemy.engine.Engine ROLLBACK
